1. Given 1000 records in a dataset, 1000 models are trained with 999 records as part of
the training sample and the remaining 1 sample for testing, and the error rate is averaged out,
this validation technique is called? 

c) LOOCV

2.  In k-fold cross validation technique, the value of k being small could lead to which of
the following in relation to the error rate

d) ?????

3. In k-fold cross validation technique, the value of k being large could lead to which of
the following in relation to the error rate

a) ?????

4. Explain what regularization is and why it is useful.

In [1]:
import boto3
import pandas as pd; pd.set_option('display.max_column', 100)
import numpy as np
import matplotlib.pyplot as plt

## define bucket in which you are trying to reach
s3 = boto3.resource('s3')
bucket_name = 'daltondencklau-data445-bucket'
bucket = s3.Bucket(bucket_name)


## define csv file to read in the bucket
file_key= 'framingham.csv'

## syntax to allow us to read the file
bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

## reading the data file
heart = pd.read_csv(file_content_stream)
heart.head(100)

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0,65,3.0,0,0.0,0.0,0,0,0,193.0,123.0,76.5,29.33,60.0,96.0,0
96,0,63,4.0,1,20.0,0.0,0,0,1,239.0,134.0,80.0,26.64,88.0,126.0,0
97,0,40,2.0,0,0.0,0.0,0,0,0,205.0,100.0,60.0,NaN,60.0,72.0,1
98,0,56,1.0,0,0.0,0.0,0,1,0,296.0,180.0,90.0,23.72,75.0,120.0,0


### Removing observations with missing values

In [2]:
heart = heart.dropna()
heart.head()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


### Changing input variables to 0-1

In [3]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
##from sklearn.preprocessing import minmax_scale
## standardizing all of the features in data set 
##heart.iloc[:, 1:15] = scaler.fit_transformation(heart.iloc[:,1:15])
##X = heart[['age', 'currentSmoker', 'totChol', 'sysBP', 'BMI', 'heartRate', 'glucose']]
##print(scaler.fit(heart))
##print(scaler.data_max)

## transforming variables
##heart_0_1_age = (heart['age'] - heart['age'].min()) / (heart['age'].max() - heart['age'].min())
##heart_0_1_totChol = (heart['totChol'] - heart['totChol'].min()) / (heart['totChol'].max() - heart['totChol'].min())
##heart_0_1_sysBP = (heart['sysBP'] - heart['sysBP'].min()) / (heart['sysBP'].max() - heart['sysBP'].min())
##heart_0_1_BMI = (heart['BMI'] - heart['BMI'].min()) / (heart['BMI'].max() - heart['BMI'].min())
##heart_0_1_heartRate = (heart['heartRate'] - heart['heartRate'].min()) / (heart['heartRate'].max() - heart['heartRate'].min())
##heart_0_1_glucose = (heart['glucose'] - heart['glucose'].min()) / (heart['glucose'].max() - heart['glucose'].min())
##heart_0_1_diaBP = (heart['diaBP'] - heart['diaBP'].min()) / (heart['diaBP'].max() - heart['diaBP'].min())

heart_0_1 = heart[['age', 'totChol', 'sysBP', 'BMI', 'heartRate', 'glucose', 'diaBP']]
heart_0_1 = scaler.fit_transform(heart_0_1)

print(heart_0_1)



#heart_x = minmax_scale(heart[['age', 'currentSmoker', 'totChol', 'sysBP', 'BMI', 'heartRate', 'glucose']])

#heart = pd.DataFrame(np.random.randm, columns = list[['age', 'currentSmoker', 'totChol', 'sysBP', 'BMI', 'heartRate', 'glucose']])


[[0.18421053 0.16837782 0.10638298 ... 0.36363636 0.10451977 0.23280423]
 [0.36842105 0.28131417 0.17730496 ... 0.51515152 0.10169492 0.34920635]
 [0.42105263 0.27104723 0.20803783 ... 0.31313131 0.08474576 0.33862434]
 ...
 [0.47368421 0.41067762 0.45153664 ... 0.22222222 0.1299435  0.46560847]
 [0.5        0.19301848 0.20330969 ... 0.21212121 0.07909605 0.33862434]
 [0.52631579 0.32032854 0.23640662 ... 0.36363636 0.18926554 0.37037037]]


### Building Model 1

In [4]:
from sklearn.model_selection import train_test_split

## defining input and target variables
X1 = heart[['age', 'currentSmoker', 'totChol', 'sysBP', 'BMI', 'heartRate', 'glucose', 'diaBP']]
Y = heart['TenYearCHD']

from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression

## defining the list to store results
k_f1score_1 = list()

kfold = KFold(n_splits = 5, shuffle = True)

for train_ix, test_ix in kfold.split(X1):

    ## splitting the data into training and validation
    X1_train, X1_val = X1.loc[train_ix], X1.loc[test_ix]
    Y_train, Y_val = Y.loc[train_ix], Y.loc[test_ix]
    
    ## building the model
    md1 = LinearRegression().fit(X1_train, Y_train)
    
    ## Predicting on the validation
    pred1 = md1.predict(X1_val)
    
    ## Storing the Prediction
    k_f1score_1.append(pred1[0])

KeyError: '[14, 26, 33, 36, 54, 72, 77, 97, 111, 114, 131, 139, 184, 193, 203, 213, 215, 216, 246, 247, 250, 263, 273, 279, 282, 293, 294, 296, 301, 305, 309, 314, 319, 338, 343, 345, 355, 382, 395, 400, 407, 412, 413, 418, 421, 425, 427, 428, 429, 433, 436, 448, 456, 467, 472, 487, 499, 503, 511, 519, 540, 552, 563, 567, 576, 577, 591, 622, 645, 661, 673, 675, 679, 689, 690, 694, 705, 719, 737, 755, 757, 760, 765, 777, 778, 782, 797, 813, 819, 822, 829, 834, 871, 872, 883, 896, 905, 917, 922, 936, 939, 942, 943, 966, 972, 980, 998, 999, 1002, 1019, 1027, 1028, 1038, 1044, 1046, 1063, 1064, 1070, 1072, 1075, 1101, 1104, 1115, 1118, 1119, 1122, 1138, 1142, 1155, 1161, 1172, 1175, 1177, 1181, 1190, 1202, 1206, 1211, 1214, 1227, 1234, 1250, 1253, 1275, 1284, 1288, 1292, 1301, 1317, 1335, 1347, 1354, 1359, 1361, 1368, 1375, 1377, 1380, 1394, 1395, 1400, 1411, 1429, 1448, 1451, 1470, 1478, 1493, 1497, 1506, 1524, 1537, 1540, 1566, 1573, 1574, 1575, 1581, 1604, 1610, 1616, 1617, 1622, 1624, 

In [ ]:
## calculating F1 score for md1

from sklearn.metrics import f1_score
md1_f1 = 2 * (precision * recall) / (precision + recall)
print (md1_f1)